---
### Optimizing Strategy Parameters
---

#### I. Load model and data

In [55]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

In [56]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df.head(n=5)

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,55.181126,55.696671,55.027369,55.443420,21220800,1.549151,UP
2016-12-09,56.049416,56.067505,55.289669,55.334891,27349400,0.321666,UP
2016-12-12,56.230289,56.347870,55.823285,55.913730,20198100,1.286169,UP
2016-12-13,56.962929,57.360890,56.293630,56.528788,35718900,-0.478644,DOWN
2016-12-14,56.691578,57.388013,56.555907,56.981005,30352700,-0.159789,DOWN


---
#### II. Simple Investment Strategy

Create Strategy and Backtest class, with specific values.

In [57]:
from backtesting import Strategy, Backtest

In [58]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [59]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

In [60]:
results = bt.run()

Interpret backtesting results.

In [61]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2025-04-03 00:00:00
Duration,3038 days 00:00:00
Exposure Time [%],91.822095
Equity Final [$],64160.337125
Equity Peak [$],71321.321388
Commissions [$],2138.229466
Return [%],541.603371


---
#### III. Parametrize the Investment Strategy

Create Strategy and Backtest class.

In [62]:
class Regression(Strategy):
    
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [63]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

Optimize backtesting with multiple combinations.

In [64]:
# [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list_limits_buy  = list(range(1, 11, 1))
# [1, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]
list_limits_sell = list(range(-1, -11, -1))

In [68]:
results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/backtesting.py:1606: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_stats.py:155: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:] / equity[:-1])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_stats.py:155: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:

---
#### III. Interpret backtesting results

`bt.optimize(..., return_heatmap=True)` returns a tuple: `(best_result, heatmap)`.

We access the second element with `results[1]`, and use `.reset_index()` to turn the MultiIndex into columns for easier manipulation.

In [69]:
df_results  = results[1].reset_index()

Pivot the `DataFrame` to create a matrix with limit_buy as rows and limit_sell as columns.
Each cell contains the `return [%]` for that parameter combination.

In [70]:
results_mtx = df_results.pivot(
    index='limit_buy',
    columns='limit_sell',
    values='Return [%]'
)

results_mtx

limit_sell,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1
limit_buy,,,,,,,,,,
1,452.680764,452.680764,516.573654,530.685047,606.509117,541.603371,888.246013,1224.078441,1785.241722,1160.524992
2,340.999309,340.999309,299.461611,308.345072,347.406801,338.063156,551.981776,194.007175,273.616277,121.039523
3,340.999309,340.999309,221.442217,180.708060,207.790236,62.213183,-81.991889,-73.698589,-64.802179,-73.207706
4,255.130013,255.130013,94.375167,69.682187,105.469198,35.540892,-88.646554,-80.938705,-78.065288,-82.493826
5,255.130013,255.130013,94.375167,69.682187,105.469198,51.082806,-88.429557,-81.409065,-78.601118,-83.994015
6,124.248840,124.248840,22.287725,9.965521,33.028262,-0.700933,-100.000000,-100.000000,-100.000000,-100.000000
7,161.400206,161.400206,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
8,146.955389,146.955389,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
9,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000


Create `DataFrame` heatmaps for better reporting.

In [71]:
results_mtx.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=results_mtx.values.min(), vmax=results_mtx.values.max())

limit_sell,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,
1,1161,1785,1224,888,542,607,531,517,453,453
2,121,274,194,552,338,347,308,299,341,341
3,-73,-65,-74,-82,62,208,181,221,341,341
4,-82,-78,-81,-89,36,105,70,94,255,255
5,-84,-79,-81,-88,51,105,70,94,255,255
6,-100,-100,-100,-100,-1,33,10,22,124,124
7,-100,-100,-100,-100,-100,-100,-100,-100,161,161
8,-100,-100,-100,-100,-100,-100,-100,-100,147,147
9,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
